In [1]:
import pdfplumber
import re
import pandas as pd
import os
import zipfile
import configparser 
from glob import glob
import tqdm
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook # benötigt conda install -c conda-forge ipywidgets
from pdfminer.pdftypes import PDFException
import requests
from zipfile import ZipFile
from zipfile import BadZipFile
import ipdb
import codecs
import numpy as np
tqdm.pandas()  # <- added this line
tqdm_notebook.pandas()

In [2]:
def creatdirection (configfile,ScratchPfad=''):
    config = configparser.ConfigParser()
    config.read(configfile, encoding='utf-8')
    if not ScratchPfad:
        ScratchPfad = config['Verzeichnisse']['verarbeitungspfad']

    Text_Pfad = os.path.join(ScratchPfad,'Text')
    PDF_Pfad = os.path.join(ScratchPfad,'PDF')  
    ZIP_Pfad = os.path.join(ScratchPfad,'ZIP')

    for pfad in (Text_Pfad,PDF_Pfad,ZIP_Pfad):
        if not os.path.exists(pfad):
            os.makedirs(pfad,exist_ok=False)
        else:
            #raise FileExistsError(f'{pfad} existiert bereits')
            print(f'{pfad} existiert bereits')
    return Text_Pfad, PDF_Pfad, ZIP_Pfad

In [5]:
Text_Pfad, PDF_Pfad, ZIP_Pfad=creatdirection('../config.ini')

C:\Temp\Stadt_SG\Text existiert bereits
C:\Temp\Stadt_SG\PDF existiert bereits
C:\Temp\Stadt_SG\ZIP existiert bereits


In [3]:
def get_traktandierte_geschaefte(configfile):
    config = configparser.ConfigParser(interpolation=None)
    config.read(configfile, encoding='utf-8')
    basedir = config['Verzeichnisse']['basedir']
    url = config['Verzeichnisse']['url']
    r = requests.get(url, allow_redirects=True)
    savepath = os.path.join(basedir,'input')
    fullfilename = os.path.join(savepath,'traktandierte-geschaefte-sitzungen-stadtparlament-stgallen.csv')
    with open(fullfilename, 'wb') as fh:
        fh.write(r.content)
    print(f'wrote to {fullfilename}.')
    inputcsv = pd.read_csv(fullfilename,sep=';')
    return inputcsv

In [6]:
inputcsv = get_traktandierte_geschaefte('../config.ini')

wrote to C:\Users\mauri\OneDrive - OST\Studium\Semester 6 (Final)\MALE\opendatasg\input\traktandierte-geschaefte-sitzungen-stadtparlament-stgallen.csv.


In [7]:
url = inputcsv['Download Traktandumsdokumente'].iloc[0]
urls = inputcsv['Download Traktandumsdokumente']
urls

0       https://ftp-sg.oca.ch/stadtparlament/43d16273e...
1       https://ftp-sg.oca.ch/stadtparlament/1b627ee96...
2       https://ftp-sg.oca.ch/stadtparlament/b4cc21efa...
3       https://ftp-sg.oca.ch/stadtparlament/eb719d563...
4       https://ftp-sg.oca.ch/stadtparlament/1b627ee96...
                              ...                        
3861    https://ftp-sg.oca.ch/stadtparlament/209495d10...
3862    https://ftp-sg.oca.ch/stadtparlament/6a3b42872...
3863    https://ftp-sg.oca.ch/stadtparlament/5d0347a5d...
3864    https://ftp-sg.oca.ch/stadtparlament/49d42124a...
3865    https://ftp-sg.oca.ch/stadtparlament/37eff2f96...
Name: Download Traktandumsdokumente, Length: 3866, dtype: object

In [8]:
def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        zf.extractall(dest_dir)


In [9]:
 def cretatedir(dest):    
    #print('create',dest)
    os.makedirs(dest,exist_ok=True)
    return dest
# Create a ZipFile Object and load sample.zip in it
def extract(zipfn,dest,verbose=False):
    try: 
        with ZipFile(zipfn, 'r') as zipfile:
           # Extract all the contents of zip file in current directory
            try:
                zipfile.extractall(path=dest)
            except OSError as err:
                print(err)
                print(f'Extraction failed for {zipfn},{dest}')
                print(zipfile.namelist())
                #ipdb.set_trace()
                """dest = dest.replace('\r\n','')
                destfiles = [s.replace('\n','').replace('\r','').replace(')','') for s in zipfile.namelist()]
                for filename,dest in zip(zipfile.namelist(),destfiles):
                    filenamenew = filename.replace('\n','').replace('\r','').replace(')','')
                    zipfile.extract(filename,path=os.path.join(dest,filenamenew))
                    print(f'extracted {filename} to {dest}')
                """
            if verbose:
                print(f'extracted to {dest}')
    except BadZipFile as err:
        print(err)
        ipdb.set_trace

def create_and_extract(zipfn,destdir,verbose=False):
    if verbose:
        print(f'creating dir {destdir}')
    dest = cretatedir(destdir)
    #print('dest',dest)
    #print('zipfn',zipfn)
    #if not os.path.exists(dest):
    extract(zipfn,dest)
import ipdb
def download_and_save(url,filename=None,verbose=False,zippath=ZIP_Pfad,pdfpath=PDF_Pfad):
    """
    >>>download_and_save(url,zippath=ZIP_Pfad,pdfpath=PDF_Pfad)
    (<Response [200]>,
 'C:\\Temp\\Stadt_SG2\\ZIP\\00e39bd219f34918a9cbc202682fd945.zip')
    """
    if filename is None:
        filename = os.path.splitext(os.path.basename(url))[0]
   # zippath = os.path.join(scratchPath,f'zip')
    zipfn=os.path.join(zippath,f'{filename}.zip')
    #print(scratchPath,filename,zipfn)
    #ipdb.set_trace()
    #os.makedirs(scratchPath,exist_ok =True)
    

#    if not os.path.exists(zippath):
#        os.makedirs(zippath,exist_ok =True)
    if not os.path.exists(zipfn):
        r = requests.get(url, allow_redirects=True)
        assert  os.path.exists(zippath),'???'
        try:
            with open(zipfn, 'wb') as fh:
                fh.write(r.content)
        except PermissionError as err:
            print('--------',err)
            ipdb.set_trace()
            return r,None
        if verbose:
            print(f'saved to {zipfn}',filename)
    else: 
        #print(zipfn,'already exists')
        r=None
    #print('scratchPath',scratchPath)
    extractpath = os.path.join(pdfpath,fr'{filename}')
    create_and_extract(zipfn,extractpath)
    return r,zipfn

In [10]:
def preprocess_Text(s):
    s = re.sub('\n *',' ',s)
    s = re.sub('  +',' ',s)
    return s
    
def process_pdf(pdffile):
    with pdfplumber.open(pdffile) as pdf:
        pagelist = [page.extract_text() for page in pdf.pages]
        Text = ' '.join(pagelist)
        s = preprocess_Text(Text)
    return s
#nur mal als Test kann später gelöscht werden:
text = process_pdf(r"C:\Temp\Stadt_SG\downloads\extract\7099d3f0fbfb4a8695bd6419e5d3700b\Interpellation.pdf")
text[:500]

'C Stadt St.Gallen Stadtrat Vorlage an den Grossen Gemeinderat vom 5. August 2003 Nr. 3599 Interpellation Interpellation Maria Huber-Kobler und Beata Studer-Lenzlinger: Vandalismus in den städtischen Bussen; schriftlich Maria Huber-Kobler und Beata Studer-Lenzlinger sowie 43 Mitunterzeichnende reichten am 27. Mai 2003 eine Interpellation: „Vandalismus in den städtischen Bussen“ ein (vgl. Beilage). Der Stadtrat beantwortet die Interpellation wie folgt: 1. Bei den Verkehrsbetrieben ist der Zugang z'

In [11]:
def process_file(pdf_fn):
    subdir = os.path.split(os.path.split(pdf_fn)[0])[1]
    output_txt_fn = pdf_fn.replace('.pdf','.txt')
    output_txt_fn =  os.path.join(Text_Pfad,subdir,os.path.basename(output_txt_fn))
    if os.path.exists(output_txt_fn):
        filesize = os.path.getsize(output_txt_fn)
    if not os.path.exists(output_txt_fn) or filesize < 100:
        try:
            text = process_pdf(pdf_fn)
        except ValueError as err:
            print(f"Error {err}")
            print(f"Datei war {pdf_fn}")
            print(err,'skipping')
            return 'valueerror'
        except PDFException as err:
            print(err)
            print(f'Exception at {pdf_fn}')
            return 'pdfexception'
        os.makedirs(os.path.join(Text_Pfad,subdir),exist_ok=True)
        #print(f'{ipdf} of {L}: {output_txt_fn}')
        os.path.split(output_txt_fn)
        with open(output_txt_fn,'w',encoding='utf-8') as fh:
            fh.write(text)
        return 'success'
    else:
        #print(filesize,output_txt_fn)
        return 'file exists'        
    
l = glob(os.path.join(PDF_Pfad,r'*\*.pdf'))
t = tqdm_notebook(l)
for pdf_fn in t:
    t.set_postfix(file=f'{pdf_fn:.<100}')
    try:
        process_file(pdf_fn)
    except FileNotFoundError as err:
        print('FEHLER:',err,f'überspringe Datei {pdf_fn}')
        continue
print(f'Done. Wrote to {Text_Pfad}')

  0%|          | 0/239 [00:00<?, ?it/s]

Done. Wrote to C:\Temp\Stadt_SG\Text


Typisches Log:

    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\1f9ecaa938b443bea854b8a32263e07b\LA Neudorf Planbeilage Bestand neu - 02.07.2021 #2.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\1f9ecaa938b443bea854b8a32263e07b\LA Neudorf Planbeilage Projekt neu - 02.07.2021 #1.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\3f1260b3a4ef4dbb98227851187c5766\Übersichtsplan, St.Leonhard-Strasse, Gäbrisstrasse - Kornhausstrasse - 27.11.2020 #1.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\42f20100cf604135863a4e271cfff36e\Übersichtsplan A4 Gübsenweg und SBB-Sitterbrücke - 01.10.2020 #3.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\5a1ae32cae5c47c7a7fc285b82707547\Gestaltungsplan; Rorschacher Strasse, Buswendeplatz; Verpflichtungskredit 16.01.2020 #1.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\6139c48a32ab4a29b4c9b6767e342a7f\Übersichtsplan; Rehetobelstrasse, Flurhofstrasse bis Haus Nr. 48; Verpflichtungskredit - 18.06.2020 #3.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\68edfa289d89437e98059b0c9fd0ab16\Teilstrassenplan; St.Leonhard-Strasse, Bushaltestelle Rosenbergstrasse - 16.06.2020 #1.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\c7d4a4ed04ed413c85b3484b63a82a83\Teilstrassenplan; Teufener Strasse, Instandstellung und Neugestaltung Davidstrasse bis Oberstrasse; Verpflichtungskredit 11.02.2020 #1.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Error not enough values to unpack (expected 2, got 1)
    Datei war D:\datasets\Stadt_SG\downloads\extract\c7d4a4ed04ed413c85b3484b63a82a83\Übersichtsplan; Teufener Strasse, Instandstellung und Neugestaltung Davidstrasse bis Oberstrasse; Verpflichtungskredit 11.02.2020 #3.pdf
    not enough values to unpack (expected 2, got 1) skipping
    Done. Wrote to D:\datasets\Stadt_SG\extracts

In [68]:
file = r'C:\Temp\Stadt_SG\downloads\extract\0ffcaafb3117404385b9fcf60739c613\Übersichtsplan, Kolumbanstrasse, Heimatstrasse bis Heiligkreuzstrasse; Verpflichtu - 24.11.2020 #1.pdf'
file = r'C:\Temp\Stadt_SG\downloads\extract\1f9ecaa938b443bea854b8a32263e07b\LA Neudorf Planbeilage Bestand neu - 02.07.2021 #2.pdf'
file = r'C:\Temp\Stadt_SG\downloads\extract\1f9ecaa938b443bea854b8a32263e07b\LA Neudorf Planbeilage Projekt neu - 02.07.2021 #1.pdf'
process_file(file)

Error not enough values to unpack (expected 2, got 1)
Datei war C:\Temp\Stadt_SG\downloads\extract\1f9ecaa938b443bea854b8a32263e07b\LA Neudorf Planbeilage Projekt neu - 02.07.2021 #1.pdf
not enough values to unpack (expected 2, got 1) skipping


'valueerror'

In [70]:
def dump_to_csv():
    filelist = glob(os.path.join(Text_Pfad,'*\*.txt'))

    data = {}
    for file in filelist:
        with codecs.open(file,'r',encoding='utf-8') as fh:
            lines = fh.readlines()
        text = ''.join(lines)
        data[file]=text
    data_dict = {os.path.basename(k):v for k,v in data.items()}
    index = np.arange(len(data_dict))
    df = pd.DataFrame(data_dict,index=index)
    return df

df = dump_to_csv()


In [ ]:
def main ():
    Text_Pfad, PDF_Pfad, ZIP_Pfad=creatdirection('../config.ini')
    inputcsv = get_traktandierte_geschaefte('../config.ini')
    inputcsv['Download Traktandumsdokumente'].dropna().progress_apply(lambda s: download_and_save(s,zippath=ZIP_Pfad,pdfpath=PDF_Pfad))
    